In [ ]:
import pandas as pd
import os
import re
import json
import base64
import requests
import time
import urllib.parse



API_KEY = 'x'  ##### Paste your API key between the quotes #####
API_VERSION = '2024-12-01-preview'  # For the most recent production release: https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release
RESOURCE_ENDPOINT = 'https://unified-api.ucsf.edu/general'  # no trailing slash--this is used by libraries as a partial URL
DEPLOYMENT_NAME = "o3-mini-2025-01-31" # change accordingly based on model 

In [ ]:
df = pd.read_csv('rationales.csv') ## csv file with rationales 


In [3]:
df.columns

Index(['user_id', 'encounter_id', 'timestamp', 'demographics_demographics',
       'summaries_summary_a', 'summaries_summary_b',
       'summaries_summary_a_type', 'summaries_summary_b_type',
       'summaries_accuracy_a', 'summaries_comp_a', 'summaries_util_a',
       'summaries_accuracy_b', 'summaries_comp_b', 'summaries_util_b',
       'summaries_best_summary', 'summaries_best_summary_type',
       'summaries_best_summary_reason', 'summaries_time_seconds', 'Rule_Out',
       'Decision_Factors', 'Column 3', 'Treatment_Plan', 'Column 5'],
      dtype='object')

In [4]:
df1 = pd.read_csv('ed_study_data.csv') ## er-reason

In [5]:
df1.columns

Index(['Unnamed: 0', 'patientdurablekey', 'encounterkey', 'ArrivalDateKey',
       'DepartureDateKeyValue', 'DepartureDateKey', 'DispositionDateKeyValue',
       'primarychiefcomplaintname', 'primaryeddiagnosisname', 'sex',
       'birthdate', 'firstrace', 'preferredlanguage',
       'highestlevelofeducation', 'maritalstatus', 'Age',
       'Discharge_Summary_Date', 'Discharge_Summary_Note_Key',
       'Progress_Note_Date', 'Progress_Note_Key', 'HP_Note_Date',
       'HP_Note_Key', 'Echo_Date', 'Echo_Key', 'Imaging_Date', 'Imaging_Key',
       'Consult_Date', 'Consult_Key', 'ED_Provider_Notes_Date',
       'ED_Provider_Notes_Key', 'ECG_Date', 'ECG_Key',
       'Discharge_Summary_Text', 'Progress_Note_Text', 'HP_Note_Text',
       'Echo_Text', 'Imaging_Text', 'Consult_Text', 'ECG_Text',
       'ED_Provider_Notes_Text', 'One_Sentence_Extracted', 'note_count',
       'Generated_Summary', 'Requested_Notes', 'ED_Presentations'],
      dtype='object')

In [6]:
# ENCOUNTER KEYS (n=72) THAT WERE TESTED AGAINST GROUND TRUTH RATIONALE FROM ER PHYSICIANS. 
encounter_ids = df['encounter_id'].tolist()
print(encounter_ids)

['D23670D21687ED', 'DCC967E0FADF81', 'D21EE393DFA95D', 'D1C21632024C65', 'D1F081CA4837A8', 'D45D44A7D6055F', 'D0F8BE1332F8FF', 'D69E680703B64C', 'D1645C6D5AEB20', 'D5C3D21165138C', 'DABFE03BB375C6', 'DEFCDA262F4653', 'D742325ACDAFBD', 'D8CD1B47E8C248', 'D7E94CEE4B2678', 'D3467EA2D79E3C', 'DCE8DF95D09B49', 'D15D1F061AFFD9', 'D58E327B175E98', 'D3E241CE3A86EA', 'DDE1CABA9A0E5A', 'DFD4AA6E28973A', 'DCBAB2F33D98AD', 'D26951C370459F', 'D6AB2DB001484A', 'D79712CAAEB0DB', 'D2E7211A602896', 'D72B2753E5FFBA', 'D1DF21D05224F9', 'D1B59F87BBA97E', 'DECB843E6930DF', 'D89176AA23B830', 'D55316A034504E', 'DCD31CA1B53665', 'D5562A81C63854', 'DCFCAEFC9A8310', 'D70798A1BC9B87', 'D44515A8FFC3AF', 'DFC154F04BE9FF', 'D785298169D956', 'D71F436BE7A6DE', 'DC2672E3DAC83E', 'DD55FFE36B780B', 'D65124F366F196', 'DF14CA84AEDDF7', 'D49F71CDBD9673', 'DB5AB74D6EE493', 'DD1453B7991C7B', 'DAC118551B93BF', 'D8F6FA00C13591', 'D1010D066107F9', 'D73B7A728A9A03', 'D21A79BB8803B6', 'D0FAF33CFECD5F', 'DA0DC2F1E9A3AA', 'D6CBE082

In [7]:
len(encounter_ids)

72

In [ ]:
encounter_keys = ['D23670D21687ED', 'DCC967E0FADF81', 'D21EE393DFA95D', 'D1C21632024C65', 'D1F081CA4837A8', 'D45D44A7D6055F', 'D0F8BE1332F8FF', 'D69E680703B64C', 'D1645C6D5AEB20', 'D5C3D21165138C', 'DABFE03BB375C6', 'DEFCDA262F4653', 'D742325ACDAFBD', 'D8CD1B47E8C248', 'D7E94CEE4B2678', 'D3467EA2D79E3C', 'DCE8DF95D09B49', 'D15D1F061AFFD9', 'D58E327B175E98', 'D3E241CE3A86EA', 'DDE1CABA9A0E5A', 'DFD4AA6E28973A', 'DCBAB2F33D98AD', 'D26951C370459F', 'D6AB2DB001484A', 'D79712CAAEB0DB', 'D2E7211A602896', 'D72B2753E5FFBA', 'D1DF21D05224F9', 'D1B59F87BBA97E', 'DECB843E6930DF', 'D89176AA23B830', 'D55316A034504E', 'DCD31CA1B53665', 'D5562A81C63854', 'DCFCAEFC9A8310', 'D70798A1BC9B87', 'D44515A8FFC3AF', 'DFC154F04BE9FF', 'D785298169D956', 'D71F436BE7A6DE', 'DC2672E3DAC83E', 'DD55FFE36B780B', 'D65124F366F196', 'DF14CA84AEDDF7', 'D49F71CDBD9673', 'DB5AB74D6EE493', 'DD1453B7991C7B', 'DAC118551B93BF', 'D8F6FA00C13591', 'D1010D066107F9', 'D73B7A728A9A03', 'D21A79BB8803B6', 'D0FAF33CFECD5F', 'DA0DC2F1E9A3AA', 'D6CBE082BFC830', 'DF37DDC3355E5A', 'D6E2E906F1AB97', 'DAD3C92955E112', 'D69D11867C0A42', 'D91D5EB4FFE064', 'D56FBFBF5556F3', 'DAF87E1C0AB440', 'D7AA42416DEC49', 'D1990815BC7265', 'D1B1383A0E6E39', 'D617C54453E260', 'D2E1B47EF7DD23', 'D05DC5B3A7D6ED', 'DFC7A9DE7EBF9F', 'D60ADDECC15B8A', 'D46F63BF2488DD']

df = df1[df1['encounterkey'].isin(encounter_keys)]

In [15]:
df = pd.read_csv('llama_rationale.csv')

In [ ]:
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI

_ = load_dotenv(find_dotenv()) 

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=RESOURCE_ENDPOINT,
)

client

In [ ]:
def predict_ror_with_dynamic_notes(row):
    # Extract all available fields
    chief_complaint = row['primarychiefcomplaintname']
    sex = row['sex']
    age = row['Age']
    one_liner = row['One_Sentence_Extracted']
    ed_presentation = row['ED_Presentations']

    # Check if necessary basic fields exist
    if pd.isna(chief_complaint):
        return None, None, None, None

    # Initialize available note types with their existence status
    available_notes = {
        'Discharge Summary': not pd.isna(row.get('Discharge_Summary_Text')),
        'Progress Notes': not pd.isna(row.get('Progress_Note_Text')),
        'H&P': not pd.isna(row.get('HP_Note_Text')),
        'Echo': not pd.isna(row.get('Echo_Text')),
        'Imaging': not pd.isna(row.get('Imaging_Text')),
        'Consult': not pd.isna(row.get('Consult_Text')),
        'ECG': not pd.isna(row.get('ECG_Text')),
    }

    # Step 1: Ask the model which notes it wants to see (always include Discharge Summary if available)
    url = f"{RESOURCE_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"

    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY
    }

    # First message to decide which notes to read
    selection_payload = {
        "messages": [
            {"role": "user", "content": "You are an experienced Emergency Department (ED) physician. Your task is to decide which medical notes you need to have a clinical understanding of before you see the patient and create your differential list."},
            {"role": "user", "content": f"Patient basic info: {age}yo {sex} with chief complaint: {chief_complaint}\n\n"
                                      f"Available notes (respond ONLY with the names of notes you want to see, separated by commas):\n"
                                      f"- Discharge Summary: {'Available' if available_notes['Discharge Summary'] else 'Not available'}\n"
                                      f"- Progress Notes: {'Available' if available_notes['Progress Notes'] else 'Not available'}\n"
                                      f"- H&P: {'Available' if available_notes['H&P'] else 'Not available'}\n"
                                      f"- Echo: {'Available' if available_notes['Echo'] else 'Not available'}\n"
                                      f"- Imaging: {'Available' if available_notes['Imaging'] else 'Not available'}\n"
                                      f"- Consult: {'Available' if available_notes['Consult'] else 'Not available'}\n"
                                      f"- ECG: {'Available' if available_notes['ECG'] else 'Not available'}\n"
                                      f"Based on the chief complaint, list ONLY the note types you need to review (comma-separated, no explanation). Always include Discharge Summary if available:"}
        ],
        # "temperature": 0.1,
        "max_completion_tokens": 1000
    }

    # Request note selection
    retries = 0
    requested_notes = []
    notes_requested_str = ""  # String to track requested notes

    while retries < 3:
        try:
            selection_response = requests.post(url, headers=headers, json=selection_payload)
            selection_response.raise_for_status()

            # Get requested note types
            notes_text = selection_response.json()["choices"][0]["message"]["content"].strip()
            requested_notes = [note.strip() for note in notes_text.split(',')]

            # Save the original request for the tracking column
            notes_requested_str = notes_text

            # Filter out unavailable notes
            requested_notes = [note for note in requested_notes
                              if note in available_notes.keys() and available_notes[note]]

            # Always include Discharge Summary if available and not already requested
            if available_notes['Discharge Summary'] and 'Discharge Summary' not in requested_notes:
                requested_notes.append('Discharge Summary')
                if notes_requested_str:
                    notes_requested_str += ", Discharge Summary (auto-added)"
                else:
                    notes_requested_str = "Discharge Summary (auto-added)"

            break
        except requests.exceptions.RequestException as e:
            print(f"Note selection request failed: {e}. Retrying {retries+1}/3...")
            time.sleep(5)
            retries += 1

    if not requested_notes and available_notes['Discharge Summary']:
        # Default to discharge summary if selection failed but it's available
        requested_notes = ['Discharge Summary']
        notes_requested_str = "Failed to get selections. Defaulted to: Discharge Summary"
    elif not requested_notes:
        # If selection failed and no discharge summary, use whatever is available
        available_notes_list = [note for note, available in available_notes.items() if available][:2]
        requested_notes = available_notes_list
        notes_requested_str = f"Failed to get selections. Defaulted to: {', '.join(available_notes_list)}"

    # Step 2: Prepare the actual notes content
    notes_content = ""

    # Map note types to DataFrame column names
    note_type_to_column = {
        'Discharge Summary': 'Discharge_Summary_Text',
        'Progress Notes': 'Progress_Note_Text',
        'H&P': 'HP_Note_Text',
        'Echo': 'Echo_Text',
        'Imaging': 'Imaging_Text',
        'Consult': 'Consult_Text',
        'ECG': 'ECG_Text',
    }
    
    # Function to truncate text to manage token limits
    def truncate_text(text, max_chars=3000):
        if text and len(text) > max_chars:
            return text[:max_chars] + "..."
        return text

    # Add requested notes to content
    for note_type in requested_notes:
        column_name = note_type_to_column.get(note_type)
        if column_name and not pd.isna(row.get(column_name)):
            notes_content += f"\n\n{note_type}:\n{truncate_text(row[column_name])}"

    # ==================== MODIFIED CODE STARTS HERE ====================
    # Instead of making one API call, we'll make three separate calls for each section
    
    # Common system message for context
    system_message = "You are an experienced Emergency Department (ED) physician tasked with creating a comprehensive assessment and plan for patients. Based on the available clinical information, you will provide: 1) A differential diagnosis list, 2) Medical decision factors including necessary exams/imaging to rule out diagnoses, and 3) A treatment plan including disposition and recommendations. Answer these questions:\n"

    # 1. First API call - DIFFERENTIAL DIAGNOSIS (without ED presentation)
    differential_system_message = system_message + " DIFFERENTIAL QUESTION: What are your initial differential diagnoses based on the past medical history (PREVIOUS HOSPITAL ENCOUNTER) and the chief complaint(CURRENT CHIEF COMPLAINT -- THIS IS THE TREATMENT FOCUS), and why? Please also address which diagnoses are lowest on your differential and why"
    
    differential_payload = {
        "messages": [
            {"role": "user", "content": f"{differential_system_message}\n"
                                       f"Based on the patient's chief complaint, age, sex, and available clinical information, provide:\n\n"
                                       f"DIFFERENTIAL DIAGNOSIS: List of differential diagnoses to consider for the CURRENT chief complaint given clinically relevant information such as the PAST medical notes.\n\n"
                                       f"IMPORTANT: Everything in the notes is from PAST encounters. The patient is NOW presenting with a NEW complaint: Chief Complaint: {chief_complaint}\n"
                                       f"Age: {age}\n"
                                       f"Sex: {sex}\n"
                                       f"Patient One-liner Summary: {one_liner}\n"
                                       f"REMINDER: All of these notes are from the PREVIOUS hospital encounter for this patient. This should only give you context for their current complaint:{notes_content}"}
        ],
        # "temperature": 0.1,
        "max_completion_tokens": 6000
    }

    # 2. Second API call - MEDICAL DECISION FACTORS (without ED presentation)
    decision_system_message = system_message + " DECISION FACTORS: What additional information or studies would you want in order to help narrow your differential diagnoses? How would you weigh their relative importance? How would you weigh their relative importance?"
    
    decision_payload = {
        "messages": [
            {"role": "user", "content": f"{decision_system_message}\n"
                                      f"Based on the patient's chief complaint, age, sex, and available clinical information, provide:\n\n"
                                      f"MEDICAL DECISION FACTORS: List what important additional information you'd like to know on history, what important exam findings you'd assess for, and what imaging/tests you would do, in addition to any formal consultations you'd made with specialists -- this should be items that would be necessary to rule out diagnoses on the differential list for the CURRENT chief complaint. NOTE: NOTHING has currently been performed.\n\n"
                                      f"IMPORTANT: Everything in the notes is from PAST encounters. The patient is NOW presenting with a NEW complaint: Chief Complaint: {chief_complaint}\n"
                                      f"Age: {age}\n"
                                      f"Sex: {sex}\n"
                                      f"Patient One-liner Summary: {one_liner}\n"
                                      f"REMINDER: All of these notes are from the PREVIOUS hospital encounter for this patient. This should only give you context for their current complaint:{notes_content}"}
        ],
        # "temperature": 0.1,
        "max_completion_tokens": 6000
    }

    # 3. Third API call - TREATMENT PLAN (WITH ED presentation)
    treatment_system_message = system_message + " TREATMENT PLAN: Given the additional history and physical exam provided in the ED provider note, which additional diagnostic tests would you like to order and what is your recommended initial treatment plan?"
    
    treatment_payload = {
        "messages": [
            {"role": "user", "content": f"{treatment_system_message}\n"
                                      f"Based on the patient's chief complaint, age, sex, and available clinical information, provide:\n\n"
                                      f"TREATMENT PLAN: Given the current history and physical provided in the ED note, what do you think is the most likely diagnosis and what is your recommended treatment plan? If you do not have enough information to conclude a final diagnosis, describe what information you still need. Include disposition (discharge, admit, ICU, etc.) and treatment recommendations for the patient given the history and physical exam in the current ER visit.\n\n"
                                      f"IMPORTANT: Everything in the notes is from PAST encounters. The patient is NOW presenting with a NEW complaint: Chief Complaint: {chief_complaint}\n"
                                      f"Age: {age}\n"
                                      f"Sex: {sex}\n"
                                      f"Patient One-liner Summary: {one_liner}\n"
                                      f"REMINDER: All of these notes are from the PREVIOUS hospital encounter for this patient. This should only give you context for their current complaint:{notes_content}"}
        ],
        # "temperature": 0.1,
        "max_completion_tokens": 6000
    }

    # Add ED Presentation ONLY to the treatment plan payload
    if pd.notna(row['ED_Presentations']):
        treatment_payload["messages"][0]["content"] += f"\nCURRENT ED Encounter Physical associated with the CURRENT CHIEF COMPLAINT: {row['ED_Presentations']}"

    # Function to make API call with retries
    def make_api_call(payload):
        retries = 0
        while retries < 3:
            try:
                response = requests.post(url, headers=headers, json=payload)
                response.raise_for_status()
                content = response.json()["choices"][0]["message"]["content"].strip()
                return content
            except requests.exceptions.RequestException as e:
                print(f"API request failed: {e}. Retrying {retries+1}/3...")
                time.sleep(5)
                retries += 1
        return None

    # Make the three separate API calls
    differential_diagnosis = make_api_call(differential_payload)
    if differential_diagnosis:
        # Ensure we have correct section header but don't add it if already there
        if not differential_diagnosis.startswith("1. DIFFERENTIAL DIAGNOSIS") and not differential_diagnosis.startswith("DIFFERENTIAL DIAGNOSIS"):
            differential_diagnosis = "1. DIFFERENTIAL DIAGNOSIS\n" + differential_diagnosis
        
    medical_decision_factors = make_api_call(decision_payload)
    if medical_decision_factors:
        # Ensure we have correct section header but don't add it if already there
        if not medical_decision_factors.startswith("2. MEDICAL DECISION FACTORS") and not medical_decision_factors.startswith("MEDICAL DECISION FACTORS"):
            medical_decision_factors = "2. MEDICAL DECISION FACTORS\n" + medical_decision_factors
        
    treatment_plan = make_api_call(treatment_payload)
    if treatment_plan:
        # Ensure we have correct section header but don't add it if already there
        if not treatment_plan.startswith("3. TREATMENT PLAN") and not treatment_plan.startswith("TREATMENT PLAN"):
            treatment_plan = "3. TREATMENT PLAN\n" + treatment_plan
            
    # Fix common parsing issues by ensuring each response is properly formatted
    # This helps when the model might include multiple sections in one response
    
    # Clean differential diagnosis
    if differential_diagnosis and ("MEDICAL DECISION FACTORS" in differential_diagnosis or "TREATMENT PLAN" in differential_diagnosis):
        parts = re.split(r'(MEDICAL DECISION FACTORS|2\.\s*MEDICAL DECISION FACTORS|TREATMENT PLAN|3\.\s*TREATMENT PLAN)', differential_diagnosis, flags=re.IGNORECASE)
        differential_diagnosis = parts[0].strip()
        
    # Clean medical decision factors
    if medical_decision_factors and ("DIFFERENTIAL DIAGNOSIS" in medical_decision_factors or "TREATMENT PLAN" in medical_decision_factors):
        parts = re.split(r'(DIFFERENTIAL DIAGNOSIS|1\.\s*DIFFERENTIAL DIAGNOSIS|TREATMENT PLAN|3\.\s*TREATMENT PLAN)', medical_decision_factors, flags=re.IGNORECASE)
        for i, part in enumerate(parts):
            if re.search(r'(MEDICAL DECISION FACTORS|2\.\s*MEDICAL DECISION FACTORS)', part, flags=re.IGNORECASE):
                if i+1 < len(parts):
                    medical_decision_factors = parts[i+1].strip()
                    break
        
    # Clean treatment plan
    if treatment_plan and ("DIFFERENTIAL DIAGNOSIS" in treatment_plan or "MEDICAL DECISION FACTORS" in treatment_plan):
        parts = re.split(r'(DIFFERENTIAL DIAGNOSIS|1\.\s*DIFFERENTIAL DIAGNOSIS|MEDICAL DECISION FACTORS|2\.\s*MEDICAL DECISION FACTORS)', treatment_plan, flags=re.IGNORECASE)
        for i, part in enumerate(parts):
            if re.search(r'(TREATMENT PLAN|3\.\s*TREATMENT PLAN)', part, flags=re.IGNORECASE):
                if i+1 < len(parts):
                    treatment_plan = parts[i+1].strip()
                    break

    return differential_diagnosis, medical_decision_factors, treatment_plan, notes_requested_str

# Apply function to DataFrame - now returns four values
results = df.apply(lambda row: predict_ror_with_dynamic_notes(row), axis=1)

# Split the results into four columns
df["LLM_Differential_Diagnosis"] = [result[0] for result in results]
df["LLM_Medical_Decision_Factors"] = [result[1] for result in results]
df["LLM_Treatment_Plan"] = [result[2] for result in results]
df["Requested_Notes"] = [result[3] for result in results]

# Save to CSV and Display
df.to_csv("o3_72_rationale.csv", index=False)